In [1]:
!pip install rasa rasa-sdk transformers torch nltk googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 34.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
!pip install googletrans==4.0.0-rc1

  Using cached googletrans-4.0.0rc1.tar.gz (20 kB)
  Preparing metadata (setup.py) ... done
  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached hstspreload-2025.1.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached h2-3.2.0-py2.py3-none-any.whl.metadata (32 kB)
  Using cached hyperframe-5.2.0-py2.py3-none-any.whl.metadata (7.2 kB)
  Using cached hpack-3.0.0-py2.py3-none-any.whl.metadata (7.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━

In [4]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
from googletrans import Translator

In [9]:
#dataset
df = pd.read_csv("school_dropout.csv")
#NLP mode
model = SentenceTransformer('all-MiniLM-L6-v2')
#embeddings
dropout_reasons = df["User"].tolist()
reason_embeddings = model.encode(dropout_reasons, convert_to_tensor=True)

In [10]:
#translator
translator = Translator()

In [11]:
supported_languages = {
    "en": "English",
    "ta": "தமிழ் (Tamil)"
}

In [12]:
#closest matching dropout reason
def match_reason(user_input):
    input_embedding = model.encode(user_input, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(input_embedding, reason_embeddings)
    best_match_idx = torch.argmax(similarities).item()
    return dropout_reasons[best_match_idx]

In [13]:
#solution
def get_solution(user_input, language="en"):
    matched_reason = match_reason(user_input)
    solution = df[df["User"] == matched_reason]["Assistant"].values
    response = solution[0] if len(solution) > 0 else "Sorry, no solution available."
    #translate the response
    if language == "ta":
        response = translator.translate(response, src="en", dest="ta").text
    return response

In [16]:
# Chatbot interaction
def chatbot_interact():
    print("\n🤖 Hello! I'm here to help you continue your education. Let's get started! 😊")
    #language options
    print("\n🌐 Choose your preferred language:")
    for lang_code, lang_name in supported_languages.items():
        print(f"  [{lang_code}] {lang_name}")
    #preferred language
    while True:
        language = input("\nType the code of your preferred language (e.g., 'ta' for Tamil, 'en' for English): ").strip().lower()
        if language in supported_languages:
            break
        print("⚠️ Invalid choice! Please enter a valid language code from the list.")
    print(f"\n🤖 {translator.translate('Welcome! Let’s start.', src='en', dest=language).text} 😊")
    #user details
    name = input(translator.translate("What is your name? ", src="en", dest=language).text)
    age = input(translator.translate("\nHow old are you? ", src="en", dest=language).text)
    place = input(translator.translate("\nWhere are you from? ", src="en", dest=language).text)
    education_level = input(translator.translate("\nTill which standard have you studied? ", src="en", dest=language).text)
    greeting = translator.translate(f"\nNice to meet you, {name}! Let's talk about your education journey. 😊", src="en", dest=language).text
    print(greeting)
    #dropout reason
    while True:
        dropout_reason = input(translator.translate("\nWhy did you drop out of school? (Type 'exit' to stop) ", src="en", dest=language).text)
        if dropout_reason.lower() == "exit":
            goodbye_msg = translator.translate("\nGoodbye! Stay positive and keep learning. Reach out anytime! 😊", src="en", dest=language).text
            print(goodbye_msg)
            break
        #selected language
        response = get_solution(dropout_reason, language)
        #chatbot response
        response_msg = translator.translate("\nBased on your reason, I suggest: ", src="en", dest=language).text + response
        print(response_msg)
        #help
        follow_up = input(translator.translate("\nWould you like to explore more options? (yes/no) ", src="en", dest=language).text)
        if follow_up.lower() not in ["yes", translator.translate("yes", src="en", dest=language).text.lower()]:
            farewell = translator.translate("\nNo problem! Keep striving for success. Goodbye! 😊", src="en", dest=language).text
            print(farewell)
            break
chatbot_interact()


🤖 Hello! I'm here to help you continue your education. Let's get started! 😊

🌐 Choose your preferred language:
  [en] English
  [ta] தமிழ் (Tamil)

Type the code of your preferred language (e.g., 'ta' for Tamil, 'en' for English): ta

🤖 வரவேற்கிறோம்!ஆரம்பிக்கலாம். 😊
உங்கள் பெயர் என்ன?Anu
உங்களுக்கு எவ்வளவு வயது?16
நீங்கள் எங்கிருந்து வருகிறீர்கள்?Coimbatore
எந்த தரத்தை நீங்கள் படித்தீர்கள்?10
உங்களை சந்தித்ததில் மகிழ்ச்சி, அனு!உங்கள் கல்வி பயணம் பற்றி பேசலாம்..
நீங்கள் ஏன் பள்ளியை விட்டு வெளியேறினீர்கள்?(நிறுத்த 'வெளியேறு' என்று தட்டச்சு செய்க)I can't afford to study
உங்கள் காரணத்தின் அடிப்படையில், நான் பரிந்துரைக்கிறேன்:தமிழ்நாடு மற்றும் கேரளா போன்ற சில மாநிலங்கள் ஈ.டபிள்யூ.எஸ் மாணவர்களுக்கு இலவச கல்லூரிக் கல்வியை வழங்குகின்றன.
மேலும் விருப்பங்களை ஆராய விரும்புகிறீர்களா?(ஆம்/இல்லை)exit
எந்த பிரச்சனையும் இல்லை!வெற்றிக்கு முயற்சி செய்யுங்கள்.குட்பை!.
